In [1]:
!pip install pandas numpy scikit-learn
!pip install sqlite3


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Importing Libraries

In [ ]:
import pandas as pd
import sqlite3
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Function 1: Extract Data from CSV

In [ ]:
def extract_data(csv_file_path):
    """
    Function to extract data from CSV file
    """
    data = pd.read_csv(csv_file_path)
    print("✅ Data extracted from CSV!")
    return data

# Function 2: Auto-Label Data using Pretrained Sentiment Analysis Model

In [ ]:
def label_data_with_model(data):
    """
    Function to use Hugging Face's pretrained sentiment analysis model to label data.
    """
    # Load a pretrained sentiment analysis pipeline
    sentiment_pipeline = pipeline("sentiment-analysis")

    # Predict sentiment for each text in the 'text' column
    predictions = sentiment_pipeline(data['text'].tolist())

    # Extract the labels from predictions and map to the DataFrame
    data['sentiment'] = [pred['label'] for pred in predictions]
    print("✅ Data labeled with sentiment using pretrained model!")
    return data

#Step 3: Transform Data into Numerical Features

In [ ]:
def transform_data(data):
    """
    Function to transform text data into numerical features using TF-IDF.
    """
    # Initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words="english")

    # Transform text data into numerical features
    X = vectorizer.fit_transform(data['text'])
    print("✅ Data transformed into numerical features using TF-IDF!")
    return X, vectorizer

# Function 4: Load Data into SQLite Database

In [ ]:
def load_data_to_sqlite(data, db_name, table_name):
    """
    Function to load the processed data into an SQLite database.
    """
    conn = sqlite3.connect(db_name)
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"✅ Data loaded into SQLite database: {db_name}, table: {table_name}!")

#  Main ETL Pipeline Execution

In [ ]:
def run_etl_pipeline(csv_file_path, db_name, table_name):
    """
    Run the full ETL pipeline to process, label, and store the data.
    """
    # Extract data
    data = extract_data(csv_file_path)

    # Label data with pretrained sentiment analysis model
    data = label_data_with_model(data)

    # Transform data (text to numerical features)
    X, vectorizer = transform_data(data)

    # Load the processed data into SQLite database
    load_data_to_sqlite(data, db_name, table_name)

#Execution:

In [4]:
csv_file_path = "/content/drive/MyDrive/sentiment_analysis.csv"  # Path to your CSV file
db_name = "sentiment_analysis.db"  # SQLite DB name
table_name = "sentiment_data"  # Table name in DB

run_etl_pipeline(csv_file_path, db_name, table_name)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


✅ Data extracted from CSV!


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

✅ Data labeled with sentiment using pretrained model!
✅ Data transformed into numerical features using TF-IDF!
✅ Data loaded into SQLite database: sentiment_analysis.db, table: sentiment_data!
